In [138]:
# Group 16
# Sjoerd Bootsma: 5242053
# Gijs van der Klink: 5389283
# Jelle Weijland: 5093457

#Version 2.0

#------------------------------------------------------------------------------------------------
#import libraries
#------------------------------------------------------------------------------------------------

import math
import pandas as pd
import os
import numpy as np
from datetime import datetime, time, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#------------------------------------------------------------------------------------------------
#datapath files
#------------------------------------------------------------------------------------------------

data_file_path = 'data/AirportData.xlsx'
data_file_path2 = 'data/Group16.xlsx'

#------------------------------------------------------------------------------------------------
# Fleet data
#------------------------------------------------------------------------------------------------

fleet_data = pd.DataFrame({
    'Type': [1, 2, 3],
    'Speed': [800, 850, 920],  # Speed in km/h
    'Cargo_Capacity': [23000, 35000, 120000],  # Cargo capacity in kg
    'TAT': [90, 120, 150],  # Average Turn-Around Time in minutes
    'Max_Range': [1500, 3300, 6300],  # Maximum range in km
    'RQ': [1400, 1600, 1800],  # Runway required in meters
    'Lease_Cost': [2143, 4857, 11429],  # Lease cost in EUR/day
    'Fixed_Cost': [750, 1500, 3125],  # Fixed operating cost per flight leg in EUR
    'Time_Cost': [1875, 1938, 3500],  # Cost per hour in EUR
    'Fuel_Cost': [2.5, 5, 9.5],  # Fuel cost parameter
    'Fleet': [2, 2, 1]  # Fleet count
    })

#------------------------------------------------------------------------------------------------
#load and process data 
#------------------------------------------------------------------------------------------------

def load_airport_data(file_path, sheet_name='Airport', output_csv_path='AirportData.csv'):
    airport_data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    airport_data_transposed = airport_data.transpose()
    airport_data_transposed.columns = airport_data_transposed.iloc[0]
    airport_data_transposed = airport_data_transposed[1:]
    airport_data_transposed.to_csv(output_csv_path, index=False)    
    return airport_data_transposed

#calculate distance between airports
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Aarde straal in kilometers
    lat1, lon1 = math.radians(lat1), math.radians(lon1)
    lat2, lon2 = math.radians(lat2), math.radians(lon2)
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    a = math.sin(delta_lat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_lon / 2)**2
    delta_sigma = 2 * math.asin(math.sqrt(a))
    return R * delta_sigma

#construct distance matrix of only FRA related airports
def calculate_distance_matrix(latitudes, longitudes):
    num_airports = len(latitudes)
    distance_matrix = [[0 for _ in range(num_airports)] for _ in range(num_airports)]
    for i in range(num_airports):
        for j in range(num_airports):
            if i != j:
                distance_matrix[i][j] = calculate_distance(latitudes[i], longitudes[i], latitudes[j], longitudes[j])
    return distance_matrix

def calculate_distance_dataframe(airport_data):
    latitudes = airport_data['Latitude (deg)'].astype(float).values
    longitudes = airport_data['Longitude (deg)'].astype(float).values
    city_names = airport_data['IATA code'].tolist()
    distance_matrix = calculate_distance_matrix(latitudes, longitudes)
    distance_df = pd.DataFrame(distance_matrix, index=city_names, columns=city_names)
    return distance_df.round(1)

def save_and_print_fra_distances(distance_df, output_csv_path='DistanceMatrix_FRA.csv'):
    if 'FRA' in distance_df.index and 'FRA' in distance_df.columns:
        fra_distances = distance_df.loc['FRA']
        filtered_distance_df = pd.DataFrame({"Distance to/from FRA": fra_distances})
        filtered_distance_df.to_csv(output_csv_path, index=True)

#proces demand excel file to csv file of FRA related inputs
def process_demand_data(file_path2, sheet_name):
    data = pd.read_excel(
        file_path2, 
        sheet_name=sheet_name, 
        header=None, 
        usecols=lambda x: x != 0  
    )
    data.columns = [f"Column_{i}" for i in range(1, data.shape[1] + 1)]
    data = data.rename(columns={"Column_1": "Departure", "Column_2": "Arrival"})
    num_time_columns = 30 
    time_columns = {f"Column_{i + 3}": i for i in range(num_time_columns)}  
    data = data.rename(columns=time_columns)
    fra_related_rows = data[
        (data["Departure"] == "FRA") | (data["Arrival"] == "FRA")
    ].reset_index(drop=True)


    fra_related_rows.to_csv("DemandData.csv", index=False)

    return fra_related_rows

distance_matrix_path = 'DistanceMatrix_FRA.csv'

#calculate operating cost based on distance and fleet data
def calculate_operating_costs(distances, fleet_data):
    total_costs_df = pd.DataFrame(index=distances.index)

    for _, row in fleet_data.iterrows():
        type_name = row['Type']
        fixed_cost = row['Fixed_Cost']
        time_cost_param = row['Time_Cost']
        fuel_cost_param = row['Fuel_Cost']
        airspeed = row['Speed']

        # Fixed operating cost 
        fixed_operating_cost = fixed_cost

        # Time-based costs 
        time_based_costs = (time_cost_param * distances / airspeed).round(2)

        # Fuel costs (in euros)
        fuel_costs = (fuel_cost_param * 1.42 *0.97*  distances / 1.5).round(2)

        # Total operating cost 
        total_costs = fixed_operating_cost + time_based_costs + fuel_costs
        total_costs_df[type_name] = total_costs.round(2)

    return total_costs_df

#calculate flight time based on distances and fleet data and output as csv file
def calculate_and_save_flight_times(fra_distances, fleet_data, output_csv_path):
    flight_times_df = pd.DataFrame(index=fra_distances.index)

    for _, row in fleet_data.iterrows():
        type_name = row['Type']
        speed = row['Speed']
        tat = row['TAT']
        
        #flight time calculation
        flight_times_df[type_name] = (
            ((((fra_distances / speed) + 0.5) * 60) + tat)
            .apply(lambda x: int(np.ceil(x / 6.0) * 6))  # averaged on multiple of 6
        )
    flight_times_df.to_csv(output_csv_path, index=True)

#execute definitions     
process_demand_data(data_file_path2, 'Group 16')
airport_data = load_airport_data(data_file_path)
distance_df = calculate_distance_dataframe(airport_data)
save_and_print_fra_distances(distance_df)
distances = pd.read_csv(distance_matrix_path, index_col=0)["Distance to/from FRA"]
operating_costs = calculate_operating_costs(distances, fleet_data)
output_csv_path = 'OperatingCosts_FRA.csv'
operating_costs.to_csv(output_csv_path, index=True)
calculate_and_save_flight_times(distances, fleet_data, 'FlightTimes_FRA.csv')


In [139]:
#------------------------------------------------------------------------------------------------
# Process created csv files 
#------------------------------------------------------------------------------------------------
df_operatingcost = pd.read_csv("OperatingCosts_FRA.csv", index_col=0)
df_operatingcost.columns = [1,2,3]
df_flight_times = pd.read_csv("FlightTimes_FRA.csv", index_col=0)
df_flight_times.columns = [1,2,3]
df_distance = pd.read_csv("DistanceMatrix_FRA.csv", index_col=0)
df_airportdata = pd.read_csv("AirportData.csv", index_col=0)
df_demandata = pd.read_csv("DemandData.csv", index_col=0)

# Reset index for demanddata file
df_demandata.reset_index(inplace=True)

#------------------------------------------------------------------------------------------------
#Define functions to calculate flight parameters 
#------------------------------------------------------------------------------------------------

#retrieve cost from csv file
def calc_cost(k, i, j):
    
    if i == 'FRA':
        
        cost = df_operatingcost.loc[j, k]
    elif j == 'FRA':
        
        cost = df_operatingcost.loc[i, k]   
    else:
        raise ValueError("Eén van de luchthavens moet Frankfurt zijn")

    return cost

#retrieve flight time from csv file
def calc_flight_time(k, i, j):
    if i == 'FRA':
        flight_time = df_flight_times.loc[j, k]
    elif j == 'FRA': 
        flight_time = df_flight_times.loc[i, k]
    else:
        raise ValueError('FRA must be in one of the flights')

    return flight_time

#retrieve distance from csv file
def calc_distance(i, j):
    if i == 'FRA':
        distance = df_distance.loc[j, 'Distance to/from FRA']
    elif j == 'FRA':
        distance = df_distance.loc[i, 'Distance to/from FRA']
    else:
        raise ValueError('FRA must be in one of the flights')
    return distance

#calculate revenue based on distance and cargoload 
def calc_revenue(distance, cargoload):
    revenue = 0.26 / 1000 * distance * cargoload #devide by 1000 as yield is in ASK/ton
    return revenue

#retrieve demand from csv file at correct timestamp
def calc_demand(timestamp, i, j, reference_time):

    # Calculate the number of 4-hour intervals from the reference_time
    time_diff = timestamp - reference_time

    # Calculate the interval index: divide by 4 hours (14400 seconds)
    interval_index = time_diff.total_seconds() // 14400  # 14400 seconds = 4 hours

    # Filter the correct row based on origin and destination
    demand_row = df_demandata[(df_demandata['Departure'] == i) & (df_demandata['Arrival'] == j)]

    # Get the base demand value for the current interval
    demand_value = demand_row.iloc[0, int(interval_index) + 2]  # +2 to account for 'Departure' and 'Arrival' columns

    # Calculate demand from previous intervals (if possible)
    demand_previous_1 = demand_row.iloc[0, int(interval_index) + 1] if interval_index > 0 else 0  # 1 column before
    demand_previous_2 = demand_row.iloc[0, int(interval_index)] if interval_index > 1 else 0  # 2 columns before

    # Calculate total demand with 20% correction from earlier columns
    total_demand = demand_value + 0.2 * demand_previous_1 + 0.2 * demand_previous_2

    return total_demand

#------------------------------------------------------------------------------------------------
# functions to meet constraints 
#------------------------------------------------------------------------------------------------

#check if flight is feasable
def check_flightfeasability(aircraft_type, origin, destination, distance, fleet_data):
    check = True

    max_range = fleet_data.loc[fleet_data['Type'] == aircraft_type, 'Max_Range'].values[0]
    min_runway_required = fleet_data.loc[fleet_data['Type'] == aircraft_type, 'RQ'].values[0]

    #check range requirement
    if max_range < distance:
        check = False

    runway_origin = df_airportdata.loc[df_airportdata['IATA code'] == origin, 'Runway (m)'].values[0]
    runway_destination = df_airportdata.loc[df_airportdata['IATA code'] == destination, 'Runway (m)'].values[0]

    #check runway requirement
    if min_runway_required > runway_origin or min_runway_required > runway_destination:
        check = False
    return check

#check if aircraft is flying on average 6 hours per day (average is taken as on day 1 there is no demand 6 hours prior to 00:00)
def check_minimum_daily_flight_time(route, min_hours_per_day=6):
    total_flight_time = 0  
    start_date = route['Departure Time'].min().date()
    end_date = route['Arrival Time'].max().date()

    # Calculate the total flight time for the entire route
    for index, row in route.iterrows():
        flight_duration = (row['Arrival Time'] - row['Departure Time']).total_seconds() / 60  # Convert to minutes
        total_flight_time += flight_duration

    num_days = (end_date - start_date).days + 1  # Total number of days in the route
    average_flight_time_per_day = (total_flight_time / 60) / num_days  # Convert to hours per day

    return average_flight_time_per_day >= min_hours_per_day


#------------------------------------------------------------------------------------------------
#calculate flight parameters 
#------------------------------------------------------------------------------------------------

def calc_flight_parameters(timestamp, origin, destination, aircraft_type):
    # Calculate distance between origin and destination
    distance = calc_distance(origin, destination)
    
    # Calculate flight time for the given aircraft type
    flight_time = calc_flight_time(aircraft_type, origin, destination)  # Assume this returns a `datetime.timedelta`
    
    # Get capacity for the given aircraft type from fleet data
    capacity = fleet_data.loc[fleet_data['Type'] == aircraft_type, 'Cargo_Capacity'].values[0]
    
    # Calculate cargoload between origin and destination 
    demand = calc_demand(timestamp, origin, destination, reference_time=datetime(2025,1,1,0,0))  
    cargoload = min(demand, capacity)
    
    # Calculate revenue based on distance and flow
    revenue = calc_revenue(distance, cargoload)
    
    # Calculate cost for the flight
    cost = calc_cost(aircraft_type, origin, destination)
    
    # Calculate profit as revenue minus cost
    profit = revenue - cost
    
    # Calculate the arrival time by adding flight_time (timedelta) to timestamp
    arrival_time = timestamp + timedelta(minutes=int(flight_time))
    
    return cost, revenue, profit, cargoload, distance, arrival_time, flight_time


#------------------------------------------------------------------------------------------------
#Update demand
#------------------------------------------------------------------------------------------------

def update_demand_data(selected_route):
    # Load the updated CSV file each time to ensure the latest version is used
    df_demand = pd.read_csv("DemandData.csv")

    # Update the global df_demandata to use the latest data in `calc_demand`
    global df_demandata
    df_demandata = df_demand

    # Iterate over each leg in the selected route
    for index, row in selected_route.iterrows():
        route = row['Route'].split(' - ')
        origin, destination = route[0], route[1]
        cargolaod = row['Cargoload (kg)']  # Use the cargo load directly from the route

        # Identify the correct time interval in the demand DataFrame based on departure time
        time_interval_index = (row['Departure Time'] - datetime(2025, 1, 1, 0, 0)).total_seconds() // 14400  # 4 hours in seconds
        time_interval_index = int(time_interval_index) + 2  # Offset for 'Departure' and 'Arrival' columns

        demand_row = df_demand[(df_demand['Departure'] == origin) & (df_demand['Arrival'] == destination)]
        if not demand_row.empty:
            demand_index = demand_row.index[0]

            # Subtract cargolaod from the current and previous intervals
            for interval_offset in range(0, -3, -1):  # Current, -1, and -2 intervals
                col_index = time_interval_index + interval_offset
                if col_index >= 2:  # Ensure valid column index
                    current_demand = df_demand.iloc[demand_index, col_index]
                    subtract_cargoload = min(cargolaod, current_demand)
                    df_demand.iloc[demand_index, col_index] -= subtract_cargoload
                    cargolaod -= subtract_cargoload

    # Overwrite the original CSV file and update the global demand DataFrame
    df_demand.to_csv("DemandData.csv", index=False)
    df_demandata = df_demand


In [140]:
#------------------------------------------------------------------------------------------------
# Define Node and Link classes
#------------------------------------------------------------------------------------------------

class Node():
    # Initializes a Node with time, airport, and default values for links, profit, and time elapsed
    def __init__(self, Time, airport):
        self.Time = Time
        self.Airport = airport
        self.inLink = None
        self.outLink = None
        self.Tot_profit = None
        self.Time_elapsed = 0

    # Adds an incoming link to the node
    def add_inLink(self, link):
        self.inLink = link

    # Adds an outgoing link to the node
    def add_outLink(self, link):
        self.outLink = link

    # Calculates total profit at this node
    def add_Profit(self):
        self.Tot_profit = self.outLink.Profit + self.outLink.To.Tot_profit

    # Updates the time elapsed for this node
    def add_Time(self, onground=False):
        if onground is True:
          self.Time_elapsed += 6
        else:
          self.Time_elapsed = self.Time_elapsed + self.outLink.To.Time_elapsed

    # String representation of the node
    def __str__(self):
        return f'Node at time: {self.Time} and airport: {self.Airport}'

    # Representation used for debugging
    def __repr__(self):
        return f'{self.Time}_{self.Airport}_{round(self.Tot_profit)}'

class Link():
    # Initializes a Link with origin, destination, distance, cost, revenue, profit, cargoload , and flighttime
    def __init__(self, origin, destination, distance, cost, revenue, profit, cargoload, flighttime):
        self.From = origin
        self.To = destination
        self.Distance = distance
        self.cost = cost
        self.revenue = revenue
        self.Profit = profit
        self.cargoload = cargoload
        self.Flighttime = flighttime


def add_time(time_current, hours, minutes=0):
    # Add the hours and minutes using timedelta
    return time_current + timedelta(hours=hours, minutes=minutes)

In [141]:
#initialize input for dynamic model
lease_costs = {1: 2143, 2: 4857, 3: 11429}

#initialize profit and aircraft used dictionaries
profit_dict = {}
aircraft_usage_dict = {}
df_airports = df_airportdata
df_airports = df_airports.set_index(df_airports.columns[0])

#initialize fleet, profit and routes
total_aircraft = {1: 2, 2:2, 3:1} 
selected_aircraft_dict = {1: 0, 2: 0, 3:0}
total_profit_aircraft = 0
routes_selected = {}
route_index = 1

#------------------------------------------------------------------------------------------------
# Big loop with dynamic programming inner loop
#------------------------------------------------------------------------------------------------

while sum(total_aircraft.values()) != 0:
  profits = {}
  routes = {}

  #apply dynamic model to all aircraft types 
  for aircraft in [1,2,3]:
    if total_aircraft[aircraft] <= 0:
      profits[aircraft] = float('-inf')
      continue
    
    aircraft_type = aircraft
    #------------------------------------------------------------------------------------------------
    # Inner loop dynamic program 
    
    # Define start and end nodes
    start_node = Node(datetime(2025, 1, 1, 0, 0), 'FRA')
    end_node = Node(datetime(2025, 1, 5, 23, 54), 'FRA')  

    # Generate a list of timestamps with 6-minute intervals
    start_time = start_node.Time  
    end_time = end_node.Time      
    lst_time = []

    current_time = start_time
    while current_time <= end_time:
        lst_time.append(current_time)
        current_time += timedelta(minutes=6)
    lst_time = lst_time[::-1] #ensure that the model starts at the end 
    lst_airports = list(df_airports.index) 

    nodes_dict = {}

    #keep track of innerloop progress
    i = 0
    for timestamp in lst_time: 
        i += 1
        if i % 100 == 0:
          print(round(i/1200*100, 2), '%')
        
        #set profit at end node at 0 
        if timestamp == end_node.Time:
            end_node.Tot_profit = 0
            end_node.Time_elapsed = 0
            end_node.add_outLink('Dummy') #there is no outlink possible at end node 
            nodes_dict[timestamp] = {i: None for i in df_airports.index}
            nodes_dict[timestamp]['FRA'] = end_node
    
        else:
            current_nodes = {i: None for i in df_airports.index}
            prev_nodes = nodes_dict[add_time(timestamp, 0, 6)]

            #aircraft stays at HUB
            for airport in lst_airports:
              if airport == 'FRA':
                  best_profit = prev_nodes['FRA'].Tot_profit
                  best_airport = 'FRA'

                  #aircraft moves to other airport
                  for airport_to in lst_airports:
                    if airport_to != 'FRA':
                        destination = airport_to
                        origin = 'FRA'
                        #calculate required parameters 
                        cost, revenue, profit, cargoload, distance, arrival_time, flight_time = calc_flight_parameters(timestamp, origin, destination, aircraft_type)
                        #check if flight is possible
                        if arrival_time > timestamp and airportcheck == True:
                          # Check if arrival_time exists in nodes_dict and airport_to exists within that specific timestamp's sub-dictionary
                          if arrival_time in nodes_dict and airport_to in nodes_dict[arrival_time] and nodes_dict[arrival_time][airport_to] is not None:
                              total_profit = profit + nodes_dict[arrival_time][airport_to].Tot_profit
                              if total_profit > best_profit:
                                  best_profit = total_profit
                                  best_airport = airport_to
                          else:
                              pass

                  #best airport is FRA, aircraft stays at HUB      
                  if best_airport == 'FRA':
                    destination_node = prev_nodes['FRA']
                    new_node = Node(timestamp, 'FRA')
                    new_link = Link(new_node, destination_node, 0, 0, 0, 0, 0, time(0,0))
                    new_node.add_outLink(new_link)
                    new_node.add_Profit()
                    if destination_node.Time_elapsed != 0: 
                      new_node.add_Time(onground=True)
                    current_nodes[airport] = new_node

                  #best airport is not FRA, aircraft moves to best airport
                  else:
                    cost, revenue, profit, cargoload, distance, arrival_time, flight_time = calc_flight_parameters(timestamp, 'FRA', best_airport, aircraft_type)
                    destination_node = nodes_dict[arrival_time][best_airport]
                    new_node = Node(timestamp, 'FRA')
                    new_link = Link(new_node, destination_node, distance, cost, revenue, profit, cargoload, flight_time)
                    new_node.add_outLink(new_link)
                    new_node.add_Profit()
                    new_node.add_Time()
                    current_nodes[airport] = new_node

              else:
                  destination = 'FRA'
                  origin = airport
                  cost, revenue, profit, cargoload, distance, arrival_time, flight_time = calc_flight_parameters(timestamp, origin, destination, aircraft_type)
                  airportcheck = check_flightfeasability(aircraft_type, origin, destination, distance, fleet_data)

                  # Check if arrival_time exists in nodes_dict, and also check if destination has a node at that time
                  if arrival_time > timestamp and airportcheck == True:
                      # Check if arrival_time exists and destination has a valid node at that time
                      if arrival_time in nodes_dict and destination in nodes_dict[arrival_time] and nodes_dict[arrival_time][destination] is not None:
                          # If the node exists, check if profit from this flight is greater than the previous one
                          if prev_nodes[airport] is None or profit + nodes_dict[arrival_time][destination].Tot_profit > prev_nodes[airport].Tot_profit:
                              destination_node = nodes_dict[arrival_time][destination]
                              new_node = Node(timestamp, airport)
                              
                              # Create a new link and add it to the new node
                              new_link = Link(new_node, destination_node, distance, cost, revenue, profit, cargoload, flight_time)
                              new_node.add_outLink(new_link)
                              new_node.add_Profit()
                              new_node.add_Time()
                              current_nodes[airport] = new_node

                  elif prev_nodes[airport] is not None:
                      destination_node = prev_nodes[airport]
                      new_node = Node(timestamp, airport)
                      new_link = Link(new_node, destination_node, 0, 0, 0, 0, 0, time(0, 0))
                      new_node.add_outLink(new_link)
                      new_node.add_Profit()
                      new_node.add_Time(onground=True)
                      current_nodes[airport] = new_node

            nodes_dict[timestamp] = current_nodes

    node = nodes_dict[timestamp]['FRA']
    #route profit per aircraft type - lease kost for 5 day period
    profits[aircraft] = node.Tot_profit - 5 * lease_costs[aircraft]
    print('Total profit of', aircraft, 'is:', profits[aircraft])

    #create output for the route
    d = {'Departure Time': [], 'Route': [], 'Arrival Time': [], 'Cost': [], 'Revenue': [], 'Profit': [], 'Flight Duration (minutes)': [], 'Cargoload (kg)': []}
    routedisplay = pd.DataFrame(d)
    routes[aircraft] = routedisplay
    
    #create route
    while node != end_node:
      nextnode = node.outLink.To
      if nextnode.Airport != node.Airport:
        departure = node.Time
        route = f'{node.Airport} - {nextnode.Airport}'
        arrival = nextnode.Time
        cost = node.outLink.cost
        revenue = node.outLink.revenue
        profit = node.outLink.Profit
        flight_duration = (nextnode.Time - node.Time).total_seconds() / 60  # Flight duration in minutes
        cargoload = node.outLink.cargoload
        routedisplay.loc[len(routedisplay.index)] = [departure, route, arrival, cost, revenue, profit, flight_duration, cargoload]
      node = nextnode

    #print created route
    print(f'\nRoute for aircraft type {aircraft}:')
    print(routes[aircraft].to_string(index=False))

    # Check if the route meets the minimum blocktime requirement of 6 hours average per day 
    if not check_minimum_daily_flight_time(routedisplay):
        print(f"Route for aircraft type {aircraft} does not meet the minimum average daily flight time of 6 hours.")
        profits[aircraft] = float('-inf') 
        continue
  #------------------------------------------------------------------------------------------------
  #Big loop continues where aircraft is selected
  #------------------------------------------------------------------------------------------------
  #select most profitable aircraft type
  ac_selected = max(profits, key=profits.get)
  print('Selected aircraft is type', ac_selected) 

  #calculate route profit
  profit = profits[ac_selected] 
  print(f'With profit of €{round(profit, 2)}')
  
  #check if the route is profitable
  if profit < 0 or profits[1] <= 0 and profits[2] <= 0 and profits[3] <= 0:
    print('No more profitable routes possible')
    break

  total_profit_aircraft += profit

  #remove selected aircraft from available aircraft types
  total_aircraft[ac_selected] -= 1
  selected_aircraft_dict[ac_selected] += 1
  print('Aircraft left:', total_aircraft)
  print('-------------- NEW ITERATION ----------------')

  route_selected = routes[ac_selected]

  #remove the transported cargoload from the demand data file
  update_demand_data(route_selected)

  routes_selected[f'{ac_selected}.{route_index}'] = [profit, route_selected]
  route_index += 1



8.33 %
16.67 %
25.0 %
33.33 %
41.67 %
50.0 %
58.33 %


KeyboardInterrupt: 

In [ ]:
#------------------------------------------------------------------------------------------------
# output data in graphs 
#------------------------------------------------------------------------------------------------

#give names to aircraft types
def get_freighter_name(ac_type):
    freighter_names = {
        1: 'Small Freighter',
        2: 'Medium Freighter',
        3: 'Large Freighter'
    }
    return freighter_names.get(ac_type, 'Unknown Freighter')

#plot timespace model 
def plot_time_space_diagram_from_routedisplay(routedisplay, route_name, save_path='plots'):
    ac_type, route_num = map(int, route_name.split('.'))  
    freighter_name = get_freighter_name(ac_type)
    title_name = f"{freighter_name} - Route {route_num}"

    airports = list(set([leg.split(' - ')[0] for leg in routedisplay['Route']] + [leg.split(' - ')[1] for leg in routedisplay['Route']]))
    airports.sort() 

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    fig, ax = plt.subplots(figsize=(12, 6))
    colors = {'Flight Arc': 'black', 'Turnaround Time': 'blue', 'Ground Time': 'green'}

    for i, row in routedisplay.iterrows():
        origin, destination = row['Route'].split(' - ')
        dep_time = row['Departure Time']
        arr_time = row['Arrival Time'] - timedelta(minutes=90)  # Adjust arrival time to exclude turnaround time as tot is included in arrival time

        # Plot flight arc (departure to arrival)
        ax.plot([dep_time, arr_time], [origin, destination], label='Flight Arc', color=colors['Flight Arc'], linewidth=0.8)
        ax.scatter([dep_time], [origin], color='blue')  # Departure point
        ax.scatter([arr_time], [destination], color='blue')  # Arrival point

        # Plot turnaround time at the destination
        turnaround_start = arr_time
        turnaround_end = turnaround_start + timedelta(minutes=90)  # 90 minutes turnaround
        ax.plot([turnaround_start, turnaround_end], [destination, destination], linestyle='dashed', label='Turnaround Time', color=colors['Turnaround Time'])

        # Plot ground before the next departure
        if i < len(routedisplay) - 1:
            next_dep_time = routedisplay.iloc[i + 1]['Departure Time']
            if next_dep_time > turnaround_end:
                ax.plot([turnaround_end, next_dep_time], [destination, destination], linestyle='dotted', label='Ground Time', color=colors['Ground Time'])

    # Format the x-axis for time
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.xticks(rotation=45)
    
    # Set labels and title
    ax.set_title(f"Time-Space Network Diagram For {title_name}")
    ax.set_xlabel("Time")
    ax.set_ylabel("Airport")
    ax.set_yticks(airports)
    
    # Customize grid lines
    ax.grid(visible=True, which='major', axis='y', color='red', linestyle=':', linewidth=0.5)  # Horizontal red dashed lines
    ax.grid(visible=False, axis='x')  # Remove vertical grid lines

    # Avoid duplicate legend entries
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='upper left')

    plt.tight_layout()

    # Save the plot as a PNG
    file_name = f"{route_name.replace(' ', '_')}.png"
    file_path = os.path.join(save_path, file_name)
    plt.savefig(file_path, format='png')
    print(f"Saved plot to {file_path}")

    plt.show()

# plot timespace model for all flights in routes selected
def plot_all_time_space_diagrams(routes_selected):
    for route_name, (profit, routedisplay) in routes_selected.items():
        print(f"Plotting Time-Space Network Diagram for {route_name}...")
        plot_time_space_diagram_from_routedisplay(routedisplay, route_name)

plot_all_time_space_diagrams(routes_selected)


In [ ]:
#------------------------------------------------------------------------------------------------
# Compute outputs for each selected route
#------------------------------------------------------------------------------------------------

def calculate_load_factor(route_df, aircraft_type, fleet_data):
    max_capacity = fleet_data.loc[fleet_data['Type'] == aircraft_type, 'Cargo_Capacity'].values[0]
    total_cargoload = route_df['Cargoload (kg)'].sum()
    total_capacity = max_capacity * len(route_df)
    load_factor = total_cargoload / total_capacity if total_capacity > 0 else 0
    return load_factor

def count_flights_to_destinations(route_df):
    destination_counts = route_df['Route'].str.split(' - ').str[1].value_counts()
    return destination_counts

def count_total_flights(route_df):
    return len(route_df)

def calculate_total_cargo_flown(route_df):
    return route_df['Cargoload (kg)'].sum()

def calculate_total_revenue(route_df):
    return route_df['Revenue'].sum()

def calculate_total_profit(route_df):
    return route_df['Profit'].sum()

def calculate_total_leasing_cost(aircraft_type, fleet_data, num_aircraft_used):
    lease_cost = fleet_data.loc[fleet_data['Type'] == aircraft_type, 'Lease_Cost'].values[0]
    return lease_cost * num_aircraft_used * 5 # for 5 days

def calculate_cost(route_df):
    return route_df['Cost'].sum()

def calculate_total_flight_time(route_df):
    total_flight_time_minutes = route_df['Flight Duration (minutes)'].sum()
    return total_flight_time_minutes

for aircraft_key, (profit, route_df) in routes_selected.items():
    aircraft_type = int(aircraft_key.split('.')[0])  # Extract aircraft type from the key
    num_aircraft_used = fleet_data.loc[fleet_data['Type'] == aircraft_type, 'Fleet'].values[0] - total_aircraft[aircraft_type]

    print(f"--- Parameters for Aircraft Type {aircraft_type} ---")
    load_factor = calculate_load_factor(route_df, aircraft_type, fleet_data)
    total_fligth_time = calculate_total_flight_time(route_df)
    destinations_count = count_flights_to_destinations(route_df)
    total_flights = count_total_flights(route_df)
    total_cargo = calculate_total_cargo_flown(route_df)
    total_revenue = calculate_total_revenue(route_df)
    total_operating_cost = calculate_cost(route_df)
    total_leasing_cost = calculate_total_leasing_cost(aircraft_type, fleet_data, num_aircraft_used)
    total_profit = calculate_total_profit(route_df) - total_leasing_cost
   
    

    # Print the KPI results
    print(f"Load Factor: {round(load_factor * 100, 2)}%")
    print(f"Total Flight time:", total_fligth_time, "minutes")
    print(f"Total aircraft Utilization:", ((total_fligth_time/3600)*100).round(2), "%")
    print("Number of Flights to Destinations:\n", destinations_count)
    print(f"Total Number of Flights: {total_flights}")
    print(f"Total Cargo Flown: {total_cargo} kg")
    print(f"Total Leasing Cost: €{total_leasing_cost}")
    print(f"Total Operating Cost: €{total_operating_cost}")
    print(f"Total Revenue: €{total_revenue}")
    print(f"Total Profit: €{total_profit}\n")
